In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/My\ Drive/TryTextAttackAgain

/content/drive/My Drive/TryTextAttackAgain


In [3]:
!ls 

BaseLoading.ipynb		      InferSent_snli_pretrained_GLOVE
corpus.py			      multi_nli_train_word_freq.csv
DefendBertGuide.ipynb		      plot_gamma_delta_tuning.ipynb
detect_adversarial.py		      preprocess_datasets
EvalBert.ipynb			      prioritized.py
freq_attacks_mnli.py		      __pycache__
freq_attacks_snli.py		      RunFull
freq_stats.py			      snli_train_word_freq.csv
InferSent.ipynb			      TextAttack
inferSentMultiModel.py		      TrainedBertModels
InferSent_multi_nli_pretrained_GLOVE  TrainNotebooks
inferSentSNLIModel.py		      wandb


In [4]:
#!git clone https://githubcom/QData/TextAttack.git

In [5]:
%cd TextAttack/

/content/drive/My Drive/TryTextAttackAgain/TextAttack


In [6]:
!pip install pyarrow>=0.16.0 --force-reinstall
!pip install transformers<3.2.0,>=3.1.0 --force-reinstall
!python3 setup.py install

ERROR: transformers 3.1.0 requires sacremoses, which is not installed.
ERROR: transformers 3.1.0 requires sentencepiece!=0.1.92, which is not installed.
ERROR: nlp 0.4.0 requires xxhash, which is not installed.
ERROR: flair 0.6.0.post1 requires bpemb>=0.3.2, which is not installed.
ERROR: flair 0.6.0.post1 requires deprecated>=1.2.4, which is not installed.
ERROR: flair 0.6.0.post1 requires ftfy, which is not installed.
ERROR: flair 0.6.0.post1 requires janome, which is not installed.
ERROR: flair 0.6.0.post1 requires konoha<5.0.0,>=4.0.0, which is not installed.
ERROR: flair 0.6.0.post1 requires langdetect, which is not installed.
ERROR: flair 0.6.0.post1 requires mpld3==0.3, which is not installed.
ERROR: flair 0.6.0.post1 requires segtok>=1.5.7, which is not installed.
ERROR: flair 0.6.0.post1 requires sentencepiece!=0.1.92, which is not installed.
ERROR: flair 0.6.0.post1 requires sqlitedict>=1.6.0, which is not installed.
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16

In [7]:
!pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.8.1          
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.2.10         
astor                         0.8.1          
astropy                       4.0.1.post1    
astunparse                    1.6.3          
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         20.1.0         
audioread                     2.1.8          
autograd                      1.3            
Babel                         2.8.0          
backcall                      0.2.0          
beautifulsoup4                4.6.3          
bert-score                    0.3.6          
bleach                        3.1.5          
blis                          0.4.

In [8]:
import textattack

ModuleNotFoundError: ignored

In [ ]:
!ls ../TrainedBertModels/roberta-base-multi_nli

# Evaluating model

### Matched

In [10]:
!textattack eval --model-from-huggingface  ../TrainedBertModels/roberta-base-multi_nli --dataset-from-nlp multi_nli --num-examples 10000 --dataset_split validation_matched --shuffle False

Traceback (most recent call last):
  File "/usr/local/bin/textattack", line 33, in <module>
    sys.exit(load_entry_point('textattack==0.2.4', 'console_scripts', 'textattack')())
  File "/usr/local/bin/textattack", line 25, in importlib_load_entry_point
    return next(matches).load()
  File "/usr/local/lib/python3.6/dist-packages/importlib_metadata/__init__.py", line 105, in load
    module = import_module(match.group('module'))
  File "/usr/lib/python3.6/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 994, in _gcd_import
  File "<frozen importlib._bootstrap>", line 971, in _find_and_load
  File "<frozen importlib._bootstrap>", line 941, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "<frozen importlib._bootstrap>", line 994, in _gcd_import
  File "<frozen importlib._bootstrap>", line 971, in _find_and_load
  

### MisMatched

In [ ]:
!textattack eval --model-from-huggingface  ../TrainedBertModels/roberta-base-multi_nli --dataset-from-nlp multi_nli --num-examples 10000 --dataset_split validation_mismatched --shuffle False

# Create frequency dictionary

In [ ]:
import string
import nltk
import nlp
import csv
import os
import math 
from textattack.shared.utils import words_from_text
import numpy as np

def get_tokenize_sentences(premises,hypothesis):
  all_sentences = []

  for i in zip(premises,hypothesis):
    st = i[0] + " " + i[1]
    tokens = words_from_text(st)
    all_sentences.append(tokens)
  
  return all_sentences

def return_multi_nli_train_dict(dataset):

  train_data = nlp.load_dataset(dataset)['train']

  train_dict = {'premise':[],'hypothesis':[]}

  perc_ = math.ceil(len(train_data)*0.15)
  print("----EVAL DATASET USING : " + str(perc_) + "\n")

  eval_indices = np.sort(np.random.RandomState(21).choice(np.arange(len(train_data)),size=perc_,))
  train_indices = np.delete(np.arange(len(train_data)),eval_indices)

  train_dict['premise'] = np.array(train_data['premise'])[train_indices] 
  train_dict['hypothesis'] = np.array(train_data['hypothesis'])[train_indices]

  return train_dict

def create_freq_corpus(dataset,csvname):


  if dataset is 'multi_nli':
   print('MULTI') 
   train_data = return_multi_nli_train_dict(dataset)

  else:

    train_data = nlp.load_dataset(dataset)['train']

  
  all_sentences = get_tokenize_sentences(train_data['premise'],train_data['hypothesis'])

  freq_dict = {}

  for sentence in all_sentences:
    for token in sentence:
      if token  in freq_dict:
        freq_dict[token] += 1
      else:
        freq_dict[token] = 1 

  print("WRITE")
  with open(csvname, 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in freq_dict.items():
      writer.writerow([key, value])
  

  return freq_dict

def load_csv(filename):

  with open(filename) as csv_file:
    reader = csv.reader(csv_file)
    freq_dict = dict(reader)
  
  return freq_dict


# Multi

In [ ]:
multi_file_name = '../multi_nli_train_word_freq.csv'
multi_dataset_name = 'multi_nli'
#multi_freq_dict = create_freq_corpus(multi_dataset_name,multi_file_name)
multi_freq_dict = load_csv(multi_file_name)

In [ ]:
len(multi_freq_dict)

105738

In [ ]:
multi_freq_dict['the']

'515381'

# To get frequency statistics

In [ ]:
import pandas as pd
from ast import literal_eval
import numpy as np
import math 

def phi_value(freq):
  return math.log(1+freq)

def get_phi_values(words,freq_dict,in_dict=False):

  if in_dict:
    return [phi_value(int(freq_dict[word])) for word in words if word in freq_dict]

  return [phi_value(int(freq_dict[word])) if word in freq_dict else 0 for word in words]


def get_attack_freq_stats(filename,freq_dict):
  
  attack_data = pd.read_csv(filename)

  premises = [i['premise'] for _, i in attack_data.iterrows()] 
  attacked_hypo = [i['attacked_hypothesis'] for _, i in attack_data.iterrows()] 
  original_hypo = [i['original_hypothesis'] for _, i in attack_data.iterrows()] 
  modifiable_indices = [literal_eval(i['modifiable_indices']) for _, i in attack_data.iterrows()] 
  modified_indices = [literal_eval(i['modified_indices']) for _, i in attack_data.iterrows()] 

  attacked_sentences = get_tokenize_sentences(premises,attacked_hypo)
  original_sentences = get_tokenize_sentences(premises,original_hypo)
  
  replaceable_words =[]
  replaced_words = []
  sub_words = []

  for i in range(len(original_sentences)):
    replaceable_words.append(np.array(original_sentences[i])[modifiable_indices[i]])
    replaced_words.append(np.array(original_sentences[i])[modified_indices[i]])
    sub_words.append(np.array(attacked_sentences[i])[modified_indices[i]])

  replaceable_words_freqs = []
  replaced_words_freqs = []
  sub_words_freqs = []
  sub_ovv_freqs = []

  for i in range(len(replaceable_words)):
    replaceable_words_freqs = np.concatenate((replaceable_words_freqs,get_phi_values(replaceable_words[i],freq_dict)))
    replaced_words_freqs = np.concatenate((replaced_words_freqs,get_phi_values(replaced_words[i],freq_dict)))
    sub_words_freqs = np.concatenate((sub_words_freqs,get_phi_values(sub_words[i],freq_dict)))
    sub_ovv_freqs = np.concatenate((sub_ovv_freqs,get_phi_values(sub_words[i],freq_dict, True)))

  print("MEAN:\t" + str(np.mean(replaceable_words_freqs)) + "\t STD:\t" + str(np.std(replaceable_words_freqs)))
  print("MEAN:\t" + str(np.mean(replaced_words_freqs)) + "\t STD:\t" + str(np.std(replaced_words_freqs)))
  print("MEAN:\t" + str(np.mean(sub_words_freqs)) + "\t STD:\t" + str(np.std(sub_words_freqs)))
  print("MEAN:\t" + str(np.mean(sub_ovv_freqs)) + "\t STD:\t" + str(np.std(sub_ovv_freqs)))

In [ ]:
!ls ../TrainedBertModels/roberta-base-multi_nli

1598359996.2828846				   mismatched
attack						   pytorch_model.bin
config.json					   README.md
events.out.tfevents.1598359996.5e941191a650.437.0  special_tokens_map.json
log.txt						   tokenizer_config.json
matched						   train_args.json
merges.txt					   vocab.json


# Attacks

##Matched

In [ ]:
!textattack attack --dataset_split validation_matched --attack_output_snli ../TrainedBertModels/roberta-base-multi_nli/matched --recipe random_nli  --num-examples 2000 --dataset-from-nlp multi_nli --model-from-huggingface ../TrainedBertModels/roberta-base-multi_nli

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-multi_nli/matched/attack/random_nli_2000_attacked.csv',multi_freq_dict)

In [ ]:
!textattack attack --dataset_split validation_matched --attack_output_snli ../TrainedBertModels/roberta-base-multi_nli/matched --attack-from-class ../prioritized Priotirized  --num-examples 2000 --dataset-from-nlp multi_nli --model-from-huggingface ../TrainedBertModels/roberta-base-multi_nli

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-multi_nli/matched/attack/prioritized_Priotirized_2000_attacked.csv',multi_freq_dict)

In [ ]:
!textattack attack --recipe textfooler --dataset_split validation_matched --attack_output_snli ../TrainedBertModels/roberta-base-multi_nli/matched  --num-examples 2000 --dataset-from-nlp multi_nli --model-from-huggingface ../TrainedBertModels/roberta-base-multi_nli

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-multi_nli/matched/attack/textfooler_2000_attacked.csv',multi_freq_dict)

In [ ]:
!textattack attack --recipe pwws_snli --dataset_split validation_matched --attack_output_snli ../TrainedBertModels/roberta-base-multi_nli/matched  --num-examples 2000 --dataset-from-nlp multi_nli --model-from-huggingface ../TrainedBertModels/roberta-base-multi_nli

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-multi_nli/matched/attack/pwws_snli_2000_attacked.csv',multi_freq_dict)

In [ ]:
!textattack attack --dataset_split validation_matched --attack_output_snli ../TrainedBertModels/roberta-base-multi_nli/matched --recipe alzantot_gpt2  --num-examples 2000 --dataset-from-nlp multi_nli --model-from-huggingface ../TrainedBertModels/roberta-base-multi_nli/

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-multi_nli/matched/attack/alzantot_gpt2_2000_attacked.csv',multi_freq_dict)

# Frequency Attacks

In [ ]:
!textattack attack --dataset_split validation_matched --attack_output_snli ../TrainedBertModels/roberta-base-multi_nli/matched/ --attack-from-class ../freq_attacks_mnli FrequencyR  --num-examples 2000 --dataset-from-nlp multi_nli --model-from-huggingface ../TrainedBertModels/roberta-base-multi_nli/

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-multi_nli/matched/attack/freq_attacks_mnli_FrequencyR_2000_attacked.csv',multi_freq_dict)

In [ ]:
!textattack attack --dataset_split validation_matched --attack_output_snli ../TrainedBertModels/roberta-base-multi_nli/matched --attack-from-class ../freq_attacks_mnli FrequencyP  --num-examples 2000 --dataset-from-nlp multi_nli --model-from-huggingface ../TrainedBertModels/roberta-base-multi_nli/

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-multi_nli/matched/attack/freq_attacks_mnli_FrequencyP_2000_attacked.csv',multi_freq_dict)

In [ ]:
!ls ../TrainedBertModels/roberta-base-multi_nli/matched/attack

## Mismatched

In [ ]:
!textattack attack --dataset_split validation_mismatched --attack_output_snli ../TrainedBertModels/roberta-base-multi_nli/mismatched --recipe random_nli  --num-examples 2000 --dataset-from-nlp multi_nli --model-from-huggingface ../TrainedBertModels/roberta-base-multi_nli

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-multi_nli/mismatched/attack/random_nli_2000_attacked.csv',multi_freq_dict)

In [ ]:
!textattack attack --dataset_split validation_mismatched --attack_output_snli ../TrainedBertModels/roberta-base-multi_nli/mismatched --attack-from-class ../prioritized Priotirized  --num-examples 2000 --dataset-from-nlp multi_nli --model-from-huggingface ../TrainedBertModels/roberta-base-multi_nli

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-multi_nli/mismatched/attack/prioritized_Priotirized_2000_attacked.csv',multi_freq_dict)

In [ ]:
!textattack attack --recipe textfooler --dataset_split validation_mismatched --attack_output_snli ../TrainedBertModels/roberta-base-multi_nli/mismatched  --num-examples 2000 --dataset-from-nlp multi_nli --model-from-huggingface ../TrainedBertModels/roberta-base-multi_nli

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-multi_nli/mismatched/attack/textfooler_2000_attacked.csv',multi_freq_dict)

In [ ]:
!textattack attack --recipe pwws_snli --dataset_split validation_mismatched --attack_output_snli ../TrainedBertModels/roberta-base-multi_nli/mismatched  --num-examples 2000 --dataset-from-nlp multi_nli --model-from-huggingface ../TrainedBertModels/roberta-base-multi_nli

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-multi_nli/mismatched/pwws_snli__2000_attacked.csv',multi_freq_dict)

In [ ]:
!textattack attack --dataset_split validation_mismatched --attack_output_snli ../TrainedBertModels/roberta-base-multi_nli/mismatched --recipe alzantot_gpt2  --num-examples 2000 --dataset-from-nlp multi_nli --model-from-huggingface ../TrainedBertModels/roberta-base-multi_nli/

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-multi_nli/mismatched/attack/alzantot_gpt2_2000_attacked.csv',multi_freq_dict)

In [ ]:
ls ../TrainedBertModels/roberta-base-multi_nli/mismatched/attack

# Frequency Attacks

In [ ]:
!textattack attack --dataset_split validation_mismatched --attack_output_snli ../TrainedBertModels/roberta-base-multi_nli/mismatched/ --attack-from-class ../freq_attacks_mnli FrequencyR  --num-examples 2000 --dataset-from-nlp multi_nli --model-from-huggingface ../TrainedBertModels/roberta-base-multi_nli/

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-multi_nli/mismatched/attack/freq_attacks_mnli_FrequencyR_2000_attacked.csv',multi_freq_dict)

In [ ]:
!textattack attack --dataset_split validation_mismatched --attack_output_snli ../TrainedBertModels/roberta-base-multi_nli/mismatched --attack-from-class ../freq_attacks_mnli FrequencyP  --num-examples 2000 --dataset-from-nlp multi_nli --model-from-huggingface ../TrainedBertModels/roberta-base-multi_nli/matched

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-multi_nli/mismatched/attack/freq_attacks_mnli_FrequencyP_2000_attacked.csv',mnli_freq_dict)

In [ ]:
!ls ../TrainedBertModels/roberta-base-multi_nli/matched/attack

In [ ]:
!ls ../TrainedBertModels/roberta-base-multi_nli/mismatched/attack

In [ ]:
!ls ../TrainedBertModels/roberta-base-multi_nli/matched/attack

# Defense Models

In [ ]:
import argparse
import copy
import importlib
import json
import os
import time
import math 

import textattack

from textattack.commands.attack.attack_args import (
    ATTACK_RECIPE_NAMES,
    BLACK_BOX_TRANSFORMATION_CLASS_NAMES,
    CONSTRAINT_CLASS_NAMES,
    GOAL_FUNCTION_CLASS_NAMES,
    HUGGINGFACE_DATASET_BY_MODEL,
    SEARCH_METHOD_CLASS_NAMES,
    TEXTATTACK_DATASET_BY_MODEL,
    WHITE_BOX_TRANSFORMATION_CLASS_NAMES,
)

# The split token allows users to optionally pass multiple arguments in a single
# parameter by separating them with the split token.
ARGS_SPLIT_TOKEN = "^"

def _cb(s):
  return textattack.shared.utils.color_text(str(s), color="blue", method="ansi")

def phi_value(freq):
    return math.log(1+freq)

def load_module_from_file(file_path):
    """Uses ``importlib`` to dynamically open a file and load an object from
    it."""
    temp_module_name = f"temp_{time.time()}"
    colored_file_path = textattack.shared.utils.color_text(
        file_path, color="blue", method="ansi"
    )
    textattack.shared.logger.info(f"Loading module from `{colored_file_path}`.")
    spec = importlib.util.spec_from_file_location(temp_module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

def parse_model(model_from_file=None, model=None,model_from_huggingface=None,model_batch_size=32):

    if model_from_file:
        # Support loading the model from a .py file where a model wrapper
        # is instantiated.
        colored_model_name = textattack.shared.utils.color_text(
            model_from_file, color="blue", method="ansi"
        )
        textattack.shared.logger.info(
            f"Loading model and tokenizer from file: {colored_model_name}"
        )
        if ARGS_SPLIT_TOKEN in model_from_file:
            model_file, model_name = model_from_file.split(ARGS_SPLIT_TOKEN)
        else:
            _, model_name = model_from_file, "model"
        try:
            model_module = load_module_from_file(model_from_file)
            print("))))HERE(((())))")
            print(model_module)
        except Exception:
            raise ValueError(f"Failed to import file {model_from_file}")
        try:
            model = getattr(model_module, model_name)
        except AttributeError:
            raise AttributeError(
                f"``{model_name}`` not found in module {model_from_file}"
            )

        if not isinstance(model, textattack.models.wrappers.ModelWrapper):
            
            if isinstance(model, textattack.models.NLINet):
                
                model = textattack.models.wrappers.PyTorchModelWrapper(
                    model, model.encoder, batch_size=model_batch_size
                )
                
            else:
                raise TypeError(
                    "Model must be of type "
                    f"``textattack.models.ModelWrapper``, got type {type(model)}"
                )

          
    elif (model in HUGGINGFACE_DATASET_BY_MODEL) or model_from_huggingface:
        # Support loading models automatically from the HuggingFace model hub.
        import transformers

        model_name = (
            HUGGINGFACE_DATASET_BY_MODEL[args.model][0]
            if (model in HUGGINGFACE_DATASET_BY_MODEL)
            else model_from_huggingface
        )

        if ARGS_SPLIT_TOKEN in model_name:
            model_class, model_name = model_name
            model_class = eval(f"transformers.{model_class}")
        else:
            model_class, model_name = (
                transformers.AutoModelForSequenceClassification,
                model_name,
            )
        colored_model_name = textattack.shared.utils.color_text(
            model_name, color="blue", method="ansi"
        )
        textattack.shared.logger.info(
            f"Loading pre-trained model from HuggingFace model repository: {colored_model_name}"
        )
        model = model_class.from_pretrained(model_name)
        tokenizer = textattack.models.tokenizers.AutoTokenizer(model_name)
        model = textattack.models.wrappers.HuggingFaceModelWrapper(
            model, tokenizer, batch_size=model_batch_size
        )
    elif model in TEXTATTACK_DATASET_BY_MODEL:
        # Support loading TextAttack pre-trained models via just a keyword.
        model_path, _ = TEXTATTACK_DATASET_BY_MODEL[model]
        model = textattack.shared.utils.load_textattack_model_from_path(
            model, model_path
        )
        # Choose the approprate model wrapper (based on whether or not this is
        # a HuggingFace model).
        if isinstance(
            model, textattack.models.helpers.BERTForClassification
        ) or isinstance(model, textattack.models.helpers.T5ForTextToText):
            model = textattack.models.wrappers.HuggingFaceModelWrapper(
                model, model.tokenizer, batch_size=model_batch_size
            )
        else:
            model = textattack.models.wrappers.PyTorchModelWrapper(
                model, model.tokenizer, batch_size=model_batch_size
            )
    else:
      raise ValueError("Nothing else supported")
    

    return model

def parse_defense_from_class(defense_from_class=None,freq_dict=None,delta=10,type='discrete',gamma='0.1',candidates=7,model_from_file=None, model=None,model_from_huggingface=None,model_batch_size=32):

    model = parse_model(model_from_file, model,model_from_huggingface,model_batch_size)
    
    if defense_from_class:
      import sys
      path = defense_from_class[0]
      path_class = path.rsplit('/')
      folder_path =  path_class[0] + '/'
      file = path_class[1]

      sys.path.insert(0,folder_path)
      module = importlib.import_module(file)
      my_class = getattr(module, defense_from_class[1])

      return my_class.build(model,freq_dict,delta=delta,type=type,gamma=gamma,candidates=candidates)
    else:
      raise ValueError("currently only attack from class is supported")

In [ ]:
import sys
sys.path.insert(0,"../")

In [ ]:
import scipy
import torch
import pandas as pd
from collections import deque
import corpus
import tqdm
import time
import textattack

logger = textattack.shared.logger

column_encoder = {"Entailment":0, "Neutral":1, "Contradiction":2}
reversed_encoder = {0:"Entailment", 1:"Neutral", 2:"Contradiction"}

def parse_log_manager():
    # Create logger
    attack_log_manager = textattack.loggers.AttackLogManager()

    '''
    # Get current time for file naming
    timestamp = time.strftime("%Y-%m-%d-%H-%M")

    # Get default directory to save results
    current_dir = os.path.dirname(os.path.realpath(__file__))
    outputs_dir = os.path.join(
        current_dir, os.pardir, os.pardir, os.pardir, "outputs", "attacks"
    )
    out_dir_txt = out_dir_csv = os.path.normpath(outputs_dir)

    # Get default txt and csv file names
    filename_txt = f"_{timestamp}.txt"
    filename_csv = f"_{timestamp}.csv"
    
    

    # in case directory doesn't exist
    if not os.path.exists(out_dir_txt):
        os.makedirs(out_dir_txt)
    if not os.path.exists(out_dir_csv):
        os.makedirs(out_dir_csv)

    attack_log_manager.add_output_file(os.path.join(out_dir_txt, filename_txt))
    '''
    #attack_log_manager.enable_visdom()
    #attack_log_manager.enable_wandb()
    attack_log_manager.enable_stdout()
    return attack_log_manager

def evaluate_set(attacked_dataset,attack,logger=True):

  num_remaining_attacks = len(attacked_dataset)
  worklist = deque(range(0, len(attacked_dataset)))
  start_time = time.time()
  worklist_tail = worklist[-1]

  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)
  #pbar = tqdm.autonotebook.tqdm(total=num_remaining_attacks)
  attack_log_manager = parse_log_manager()


  num_results = 0
  num_failures = 0
  num_successes = 0

  load_time = time.time()

  probabilities = []
  for result in attack.attack_dataset(attacked_dataset, indices=worklist):

    #or_label = result.original_result.output
    #initial_score = result.original_result.raw_output[or_label]
    #pertubated_score = result.perturbed_result.raw_output[or_label]
    initial_score = 1-result.original_result.score
    pertubated_score = 1-result.perturbed_result.score
    probabilities.append(initial_score-pertubated_score)

    if logger:
      attack_log_manager.log_result(result)

      print("\n")
    pbar.update(1)
    
    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )
    
  pbar.close()

  # Enable summary stdout
  
  if logger:
    attack_log_manager.log_summary()
    attack_log_manager.flush()
  print()
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  return num_successes, num_failures, probabilities



class EvalDefenseModel():

  def __init__(
      self,
      defense_from_class,
      model_from_file=None, 
      model=None,
      model_from_huggingface=None,
      model_batch_size=32,
      delta = 10,
      gamma = 1,
      type = 'discrete',
      freq_dict = None,
      data_file = None,
      data_name = "multi_nli",
      candidates = 7,
      logger = True,
      only_attack = False,
      only_original = False,
      #indices_to_modify=None,
      #shifted_idxs=True,
    ):

    # Get attack
    attack = parse_defense_from_class(defense_from_class,freq_dict,delta,type,gamma,candidates,model_from_file, model,model_from_huggingface,model_batch_size)
    
    #Get datasets
    attacked_data = pd.read_csv(data_file,usecols=["premise",'attacked_hypothesis', 'modified_indices' , "attacked_label"],keep_default_na=False)
    original_data = pd.read_csv(data_file,usecols=["premise",'original_hypothesis', "original_label"],keep_default_na=False)
    or_labels = original_data["original_label"]

    to_remove = []
    for index,x in attacked_data.iterrows():
      if x['modified_indices'] == '[]' or or_labels[index]==x['attacked_label']:
        to_remove.append(index)
    attacked_data = attacked_data.drop(to_remove)
    attacked_data = attacked_data.drop(['modified_indices'],axis=1)

    attacked_data["attacked_label"] = [column_encoder[i] for i in attacked_data["attacked_label"] ]


    original_data["original_label"] = [column_encoder[i] for i in original_data["original_label"] ]

    attacked_data = attacked_data.rename(columns={'attacked_hypothesis': 'hypothesis','attacked_label':'label'})
    attacked_dataset = nlp.Dataset.from_pandas(attacked_data)

    attacked_dataset = textattack.datasets.HuggingFaceNlpDataset(
            name=data_name, shuffle=False, split=None, dataset=attacked_dataset, label_map = reversed_encoder)
    
    original_data = original_data.rename(columns={'original_hypothesis': 'hypothesis','original_label':'label'})
    original_dataset = nlp.Dataset.from_pandas(original_data)

    original_dataset = textattack.datasets.HuggingFaceNlpDataset(
            name=data_name, shuffle=False, split=None, dataset=original_dataset, label_map = reversed_encoder)
    
    print("------Attacked Dataset------")
    if not only_original:
      true_positives, false_negatives, pert_probs = evaluate_set(attacked_dataset,attack,logger)
      self.tp = true_positives
      self.fn = false_negatives
      self.adv_probs = pert_probs
      print("True positives :" + str(true_positives) + "\n")
      print("False_negatives :" + str(false_negatives) + "\n")

    if  not only_attack:
      print("------Original Dataset------")
      false_positives, true_negatives, or_probs = evaluate_set(original_dataset,attack,logger)
      print("False positives :" + str(false_positives) + "\n")
      print("True negative :" + str(true_negatives) + "\n")
      self.fp = false_positives
      self.tn = true_negatives
      self.or_probs = or_probs

# Attack whole eval with random

In [ ]:
def return_multi_nli_eval_dict(dataset):

  # Get only 20 000 only
  train_data = nlp.load_dataset(dataset)['train']

  valid_dict = {'premise':[],'hypothesis':[]}

  perc_ = math.ceil(len(train_data)*0.15)
  print("----EVAL DATASET USING : " + str(perc_) + "\n")

  eval_indices = np.sort(np.random.RandomState(21).choice(np.arange(len(train_data)),size=perc_,))

  valid_dict['premise'] = np.array(train_data['premise'])[eval_indices[0:20000]] 
  valid_dict['hypothesis'] = np.array(train_data['hypothesis'])[eval_indices[0:20000]]
  valid_dict['label'] = np.array(train_data['label'])[eval_indices[0:20000]]


  return valid_dict

In [ ]:
# Create Eval datasets

multi_eval_dataset = return_multi_nli_eval_dict("multi_nli")


Dataset multi_nli downloaded and prepared to /root/.cache/huggingface/datasets/multi_nli/plain_text/1.0.0/60774175381b9f3f1e6ae1028229e3cdb270d50379f45b9f2c01008f50f09e6b. Subsequent calls will reuse this data.
----EVAL DATASET USING : 58906



In [ ]:
from textattack.shared.utils import get_modifiable_indices

def parse_attack_from_recipe(recipe=None,model_from_file=None, model=None,model_from_huggingface=None,model_batch_size=32):

  model = parse_model(model_from_file, model,model_from_huggingface,model_batch_size)

  recipe = eval(f"{ATTACK_RECIPE_NAMES[recipe]}.build(model)")
  recipe.goal_function.query_budget = float("inf")
  recipe.goal_function.model_batch_size = model_batch_size
  recipe.goal_function.model_cache_size = 2 ** 18
  recipe.constraint_cache_size = 2 ** 18
  return recipe
    
def create_snli_dict():
    dictionary = {}
    dictionary["premise"] = []
    dictionary["original_hypothesis"] = []
    dictionary["attacked_hypothesis"] = []
    dictionary["label"] = []
    dictionary["original_label"] = []
    dictionary["attacked_label"] = []
    dictionary['modifiable_indices'] = []
    dictionary['modified_indices'] = []

    return dictionary

def create_csv_file(dictionary,filename):
    df = pd.DataFrame(dictionary)
    df.to_csv(filename, encoding='utf-8', index=False)
    return df

def attack_recipe(recipe,dataset,data_name,folder,model_from_file=None, model=None,model_from_huggingface=None,model_batch_size=32):
     
  # Get attack
  attack = parse_attack_from_recipe(recipe,model_from_file, model,model_from_huggingface,model_batch_size)
    
  #Remove untified instances
  to_remove = []
  for index,x in dataset.iterrows():
    if x['label'] == -1:
      to_remove.append(index)

  dataset = dataset.drop(to_remove)
  dataset = nlp.Dataset.from_pandas(dataset)

  dataset = textattack.datasets.HuggingFaceNlpDataset(
          name=data_name, shuffle=False, split=None, dataset=dataset, label_map = reversed_encoder)
    
  num_remaining_attacks = len(dataset)
  worklist = deque(range(0, len(dataset)))
  start_time = time.time()
  worklist_tail = worklist[-1]

  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)
  attack_log_manager = parse_log_manager()


  num_results = 0
  num_failures = 0
  num_successes = 0

  load_time = time.time()

  output_attack_dir = os.path.join(folder, "attack")
  output_attack_dir = os.path.normpath(output_attack_dir)
  if not os.path.exists(output_attack_dir):
    os.makedirs(output_attack_dir)

  print("\nAttacks will be saved at " + output_attack_dir + "\n")
  attacked_dict = create_snli_dict()

  for result in attack.attack_dataset(dataset, indices=worklist):

    modifiable_indices = get_modifiable_indices(result.original_result.attacked_text,attack.pre_transformation_constraints)
    prem = result.original_result.attacked_text._text_input['premise']
    hypothesis = result.original_result.attacked_text._text_input['hypothesis']
    attacked_dict['premise'].append(prem)
    attacked_dict['original_hypothesis'].append(hypothesis)
    result.change_word_numbers = len(hypothesis.split())
    attacked_dict['label'].append(result.original_result.ground_truth_output)
    attacked_dict['original_label'].append(result.original_result._processed_output[0])
    attacked_dict['modifiable_indices'].append(list(modifiable_indices))

    if isinstance(result, textattack.attack_results.SkippedAttackResult):
        attacked_dict['attacked_hypothesis'].append(hypothesis)
        attacked_dict['attacked_label'].append(result.original_result._processed_output[0])
        attacked_dict['modified_indices'].append([])

    else:

        hypothesis = result.perturbed_result.attacked_text._text_input['hypothesis']
        attacked_dict['attacked_hypothesis'].append(hypothesis)
        attacked_dict['attacked_label'].append(result.perturbed_result._processed_output[0])
        attacked_dict['modified_indices'].append(list(result.perturbed_result.attacked_text.attack_attrs['modified_indices']))

   
    #attack_log_manager.log_result(result)

    #print("\n")
    pbar.update(1)
    
    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

  pbar.close()

  # Enable summary stdout
  
  attack_log_manager.log_summary()
  attack_log_manager.flush()
  print()
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  filename = output_attack_dir + "/" + recipe + "eval_whole" 

  create_csv_file(attacked_dict,filename+"_attacked.csv")

In [ ]:
multi_eval_dataset = pd.DataFrame.from_dict(multi_eval_dataset)
classifier = '../TrainedBertModels/roberta-base-multi_nli/'
attack_recipe("random_nli",multi_eval_dataset,"multi_nli",classifier,model_from_file=None, model=None,model_from_huggingface=classifier,model_batch_size=32)


textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/roberta-base-multi_nli/
textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


-RANDOM NOTHING TO INTIALIZE-


textattack: Loading nlp dataset multi_nli, split None.
  0%|          | 0/20000 [00:00<?, ?it/s]


Attacks will be saved at ../TrainedBertModels/roberta-base-multi_nli/attack



Exception ignored in: <bound method Widget.__del__ of HBox(children=(FloatProgress(value=1.0, bar_style='info', max=1.0, style=ProgressStyle(description_width='initial')), HTML(value=' 392702/0 [00:11&lt;00:00, 36597.26 examples/s]')))>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ipywidgets/widgets/widget.py", line 417, in __del__
    def __del__(self):
KeyboardInterrupt
[Succeeded / Failed / Total] 33 / 243 / 277:   1%|▏         | 277/20000 [00:16<19:27, 16.89it/s]

#Get prectiles for delta tuning

In [ ]:
multi_dataset = nlp.load_dataset("multi_nli")
multi_train_dataset = return_multi_nli_train_dict("multi_nli")
multi_eval_dataset = return_multi_nli_eval_dict("multi_nli")
percentiles = [0,10,20,30,40,50,60,70,80,90,100]

----EVAL DATASET USING : 58906

----EVAL DATASET USING : 58906



In [ ]:
# MNLI

frequencies_train = []
for sentence in multi_train_dataset['hypothesis']:
  words = words_from_text(sentence)
  phi_values = get_phi_values(words,multi_freq_dict)
  frequencies_train += phi_values

print("MNLI frequency precentiles")
mnli_precentiles = np.percentile(frequencies_train,percentiles)
print(mnli_precentiles)


MNLI frequency precentiles
[ 0.69314718  4.73619845  6.12905021  7.01301579  7.81883244  8.79270146
  9.81536647 10.5948832  11.3667661  12.44278374 13.15266365]


# Tune delta for discete

In [ ]:
!ls 

'=0.16.0'	    docs	 README.md	    textattack
 build		    examples	 requirements.txt   textattack.egg-info
 CONTRIBUTING.md    LICENSE	 setup.cfg	    wandb
 date_now-log.txt   Makefile	 setup.py
 dist		    pytest.ini	 tests


In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

print('Try loading Mutli training Corpus')
multi_freq_dict = corpus.load_csv('../multi_nli_train_word_freq.csv')

def boostrap_auc(or_probs,adv_probs,times):

  np.random.seed(21)
  labels = np.concatenate((np.ones((len(adv_probs),)),np.zeros((len(adv_probs),))))
  au_scores = np.zeros(times)
  for i in range(times):
    or_sample_probs = np.random.choice(or_probs,len(adv_probs))
    probs = np.concatenate((np.array(adv_probs),np.array(or_sample_probs)))
    au_scores[i] = roc_auc_score(labels,probs)
  
  return np.mean(au_scores)

Try loading Mutli training Corpus


In [ ]:
tps_discrete = []
fns_discrete = []
fps_discrete = []
tns_discrete = []
f1_discrete = []
auc_scores = []

for i in mnli_precentiles:
  print(str(i) + " delta \n")

  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-multi_nli/',
        model_batch_size=32,
        delta = i,
        type = 'discrete',
        gamma = 1,
        freq_dict = multi_freq_dict,
        data_file='../TrainedBertModels/roberta-base-multi_nli/attack/random_nlieval_whole_attacked.csv',
        data_name = 'multi_nli',
        candidates =7,
        logger = False,
        only_attack = False)
  
  tps_discrete.append(stat.tp)
  fns_discrete.append(stat.fn)
  fps_discrete.append(stat.fp)
  tns_discrete.append(stat.tn)

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_discrete.append(f1)
  
  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,1000)
  print("AUC score " + str(auc_score) + '\n')
  auc_scores.append(auc_score)
  
  

In [ ]:
print("TPS")
print(tps_discrete)
print("\nFNS")
print(fns_discrete)
print("\nFPS")
print(fps_discrete)
print("\nTNS")
print(tns_discrete)
print("\nF1scores")
print(f1_discrete)
print("\nAUCscores")
print(auc_scores)

TPS


NameError: ignored

In [ ]:
discrete_fp_rates = [fp / (fp+tn)  for fp,tn in zip(fps_discrete,tns_discrete)]
print("FPrate")
print(discrete_fp_rates)

discrete_tp_rates = [tp / (tp+fn)  for tp,fn in zip(tps_discrete,fns_discrete)]
print("\nTPrate")
print(discrete_tp_rates)
discrete_max_tp = np.argmax(discrete_tp_rates)
print("Maximum Tprate at delta :")
print(discrete_max_tp)
print("Best delta Value : ")
discrete_delta = mnli_precentiles[discrete_max_tp]
print(discrete_delta)

# Tune gamma (this will be done manually) take the best delta and try values for gamma then check the fpr

In [ ]:
discrete_delta = 9.81536646914095

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

print('Try loading MULTI training Corpus')
multi_freq_dict = corpus.load_csv('../multi_nli_train_word_freq.csv')

tps_discrete = []
fns_discrete = []
fps_discrete = []
tns_discrete = []
f1_discrete = []



for i in [0,0.2,0.4,0.6,0.8,1]:
  print(str(i) + " gamma \n")

  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-multi_nli/',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'continuous',
        gamma = i,
        freq_dict = multi_freq_dict,
        data_file='../TrainedBertModels/roberta-base-multi_nli/attack/random_nlieval_whole_attacked.csv',
        data_name = 'multi_nli',
        candidates =7,
        logger = False,
        only_attack = False,
        only_original=True)
  
  #tps_discrete.append(stat.tp)
  #fns_discrete.append(stat.fn)
  fps_discrete.append(stat.fp)
  tns_discrete.append(stat.tn)

  #true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  #predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  #f1 = f1_score(true_labels, predictions)
  #print("F1 score " + str(f1) + '\n')
  #f1_discrete.append(f1)

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

print('Try loading Multi training Corpus')
multi_freq_dict = corpus.load_csv('../multi_nli_train_word_freq.csv')


Try loading Multi training Corpus


In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

print('Try loading Multi training Corpus')
multi_freq_dict = corpus.load_csv('../multi_nli_train_word_freq.csv')

tps_discrete = []
fns_discrete = []
fps_discrete = []
tns_discrete = []
f1_discrete = []



for i in [0.825,0.85,0.875,0.9,0.925,0.95,0.975]:
  print(str(i) + " gamma \n")

  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-multi_nli/',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'continuous',
        gamma = i,
        freq_dict = multi_freq_dict,
        data_file='../TrainedBertModels/roberta-base-multi_nli/attack/random_nlieval_whole_attacked.csv',
        data_name = 'mutli_nli',
        candidates =7,
        logger = False,
        only_attack = False,
        only_original=True)
  
  #tps_discrete.append(stat.tp)
  #fns_discrete.append(stat.fn)
  fps_discrete.append(stat.fp)
  tns_discrete.append(stat.tn)

  #true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  #predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  #f1 = f1_score(true_labels, predictions)
  #print("F1 score " + str(f1) + '\n')
  #f1_discrete.append(f1)

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

print('Try loading Multi training Corpus')
multi_freq_dict = corpus.load_csv('../multi_nli_train_word_freq.csv')

tps_discrete = []
fns_discrete = []
fps_discrete = []
tns_discrete = []
f1_discrete = []



for i in [0.985]:
  print(str(i) + " gamma \n")

  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-multi_nli/',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'continuous',
        gamma = i,
        freq_dict = multi_freq_dict,
        data_file='../TrainedBertModels/roberta-base-multi_nli/attack/random_nlieval_whole_attacked.csv',
        data_name = 'mutli_nli',
        candidates =7,
        logger = False,
        only_attack = False,
        only_original=True)
  
  #tps_discrete.append(stat.tp)
  #fns_discrete.append(stat.fn)
  fps_discrete.append(stat.fp)
  tns_discrete.append(stat.tn)

  #true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  #predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  #f1 = f1_score(true_labels, predictions)
  #print("F1 score " + str(f1) + '\n')
  #f1_discrete.append(f1)

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

print('Try loading Multi training Corpus')
multi_freq_dict = corpus.load_csv('../multi_nli_train_word_freq.csv')

tps_discrete = []
fns_discrete = []
fps_discrete = []
tns_discrete = []
f1_discrete = []



for i in [0.990,0.995]:
  print(str(i) + " gamma \n")

  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-multi_nli/',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'continuous',
        gamma = i,
        freq_dict = multi_freq_dict,
        data_file='../TrainedBertModels/roberta-base-multi_nli/attack/random_nlieval_whole_attacked.csv',
        data_name = 'mutli_nli',
        candidates =7,
        logger = False,
        only_attack = False,
        only_original=True)
  
  #tps_discrete.append(stat.tp)
  #fns_discrete.append(stat.fn)
  fps_discrete.append(stat.fp)
  tns_discrete.append(stat.tn)

  #true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  #predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  #f1 = f1_score(true_labels, predictions)
  #print("F1 score " + str(f1) + '\n')
  #f1_discrete.append(f1)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/roberta-base-multi_nli/


Try loading Multi training Corpus
0.99 gamma 



textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:10<00:00, 44.0MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp_ehl3gd0.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.
textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH continuous--


textattack: Loading nlp dataset mutli_nli, split None.
textattack: Loading nlp dataset mutli_nli, split None.
  0%|          | 0/20000 [00:00<?, ?it/s]

...

--Detection SEARCH continuous--


textattack: Loading nlp dataset mutli_nli, split None.
textattack: Loading nlp dataset mutli_nli, split None.
[Succeeded / Failed / Total] 0 / 3 / 3:   0%|          | 3/20000 [00:00<18:00, 18.51it/s]

------Attacked Dataset------
------Original Dataset------


[Succeeded / Failed / Total] 744 / 19256 / 20000: 100%|██████████| 20000/20000 [17:56<00:00, 18.58it/s]
textattack: Attack time: 1076.6715533733368s



False positives :744

True negative :19256



In [ ]:
gamma_5 = 0.9875
gamma_10 = 0.8375 

# Defened against all attacks

In [ ]:
!ls ../TrainedBertModels/roberta-base-multi_nli/attack

random_nlieval_whole_attacked.csv


In [ ]:
attacks = ["random_nli_2000_attacked.csv","prioritized_Priotirized_2000_attacked.csv","textfooler_2000_attacked.csv","pwws_snli_2000_attacked.csv","alzantot_gpt2_2000_attacked.csv"]

## Matched

### Discrete

In [ ]:
tpr_discrete = []
fpr_discrete = []
f1_discrete = []
auc_scores = []


for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-multi_nli/',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'discrete',
        gamma = None,
        freq_dict = multi_freq_dict,
        data_file='../TrainedBertModels/roberta-base-multi_nli/matched/attack/' + attack,
        data_name = 'mutli_nli',
        candidates =7,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_discrete.append(stat.tp/(stat.tp+stat.fn))
  fpr_discrete.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_discrete.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_scores.append(auc_score)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/roberta-base-multi_nli/


Attack From
random_nli_2000_attacked.csv




textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH discrete--


textattack: Loading nlp dataset mutli_nli, split None.
textattack: Loading nlp dataset mutli_nli, split None.
[Succeeded / Failed / Total] 2 / 2 / 4:   1%|▏         | 4/299 [00:00<00:12, 23.64it/s]

...

------Attacked Dataset------


[Succeeded / Failed / Total] 715 / 419 / 1134: 100%|██████████| 1134/1134 [01:03<00:00, 17.79it/s]
textattack: Attack time: 63.73208260536194s
[Succeeded / Failed / Total] 0 / 3 / 3:   0%|          | 3/2000 [00:00<02:02, 16.36it/s]


True positives :715

False_negatives :419

------Original Dataset------


[Succeeded / Failed / Total] 346 / 1654 / 2000: 100%|██████████| 2000/2000 [01:48<00:00, 18.48it/s]
textattack: Attack time: 108.24759936332703s



False positives :346

True negative :1654

F1 score 0.8121777559538423

AUC score 0.7266637039292168



In [ ]:
print("TPrate")
print(tpr_discrete)
print("\nFPrate")
print(fpr_discrete)
print("\nF1 score")
print(f1_discrete)
print("\n AUC scores")
print(auc_scores)

TPrate
[0.6622073578595318, 0.6490939044481054, 0.61375, 0.6834381551362684, 0.6305114638447972]

FPrate
[0.173, 0.173, 0.173, 0.173, 0.173]

F1 score
[0.8809587217043942, 0.8554434962503231, 0.7743445692883895, 0.8054541027514001, 0.8121777559538423]

 AUC scores
[0.7086330292726032, 0.7251711342682433, 0.6763934862890625, 0.7716830276335958, 0.7266637039292168]


### Continuous 10%

In [ ]:
tpr_cont10 = []
fpr_cont10 = []
f1_cont10 = []
auc_cont10 = []


for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-multi_nli/',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'continuous',
        gamma = gamma_10,
        freq_dict = multi_freq_dict,
        data_file='../TrainedBertModels/roberta-base-multi_nli/matched/attack/' + attack,
        data_name = 'multi_nli',
        candidates =7,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_cont10.append(stat.tp/(stat.tp+stat.fn))
  fpr_cont10.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_cont10.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_cont10.append(auc_score)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/roberta-base-multi_nli/


Attack From
random_nli_2000_attacked.csv




textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH continuous--


textattack: Loading nlp dataset multi_nli, split None.
textattack: Loading nlp dataset multi_nli, split None.
[Succeeded / Failed / Total] 1 / 3 / 4:   1%|▏         | 4/299 [00:00<00:12, 23.06it/s]

...

------Attacked Dataset------


[Succeeded / Failed / Total] 383 / 751 / 1134: 100%|██████████| 1134/1134 [01:01<00:00, 18.47it/s]
textattack: Attack time: 61.3950719833374s
[Succeeded / Failed / Total] 0 / 3 / 3:   0%|          | 3/2000 [00:00<01:55, 17.32it/s]


True positives :383

False_negatives :751

------Original Dataset------


[Succeeded / Failed / Total] 230 / 1770 / 2000: 100%|██████████| 2000/2000 [01:47<00:00, 18.62it/s]
textattack: Attack time: 107.42714858055115s



False positives :230

True negative :1770

F1 score 0.783012607830126

AUC score 0.7266637039292168



In [ ]:
print("TPrate")
print(tpr_cont10)
print("\nFPrate")
print(fpr_cont10)
print("\nF1 score")
print(f1_cont10)
print("\n AUC scores")
print(auc_cont10)

TPrate
[0.3110367892976589, 0.35584843492586493, 0.2475, 0.4416491963661775, 0.3377425044091711]

FPrate
[0.115, 0.115, 0.115, 0.115, 0.115]

F1 score
[0.8903420523138833, 0.8507570295602018, 0.7117008443908324, 0.7747866053841103, 0.783012607830126]

 AUC scores
[0.7086330292726032, 0.7251711342682433, 0.6763934862890625, 0.7716830276335958, 0.7266637039292168]


## Continuous 5%

In [ ]:
tpr_cont5 = []
fpr_cont5 = []
f1_cont5 = []
auc_cont5 = []


for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-multi_nli/',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'continuous',
        gamma = gamma_5,
        freq_dict = multi_freq_dict,
        data_file='../TrainedBertModels/roberta-base-multi_nli/matched/attack/' + attack,
        data_name = 'multi_nli',
        candidates =7,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_cont5.append(stat.tp/(stat.tp+stat.fn))
  fpr_cont5.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_cont5.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_cont5.append(auc_score)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/roberta-base-multi_nli/


Attack From
random_nli_2000_attacked.csv




textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH continuous--


textattack: Loading nlp dataset multi_nli, split None.
textattack: Loading nlp dataset multi_nli, split None.
[Succeeded / Failed / Total] 0 / 4 / 4:   1%|▏         | 4/299 [00:00<00:12, 24.17it/s]

...

------Attacked Dataset------


[Succeeded / Failed / Total] 127 / 1007 / 1134: 100%|██████████| 1134/1134 [01:06<00:00, 17.07it/s]
textattack: Attack time: 66.43378829956055s
[Succeeded / Failed / Total] 0 / 3 / 3:   0%|          | 3/2000 [00:00<02:03, 16.16it/s]


True positives :127

False_negatives :1007

------Original Dataset------


[Succeeded / Failed / Total] 95 / 1905 / 2000: 100%|██████████| 2000/2000 [01:56<00:00, 17.21it/s]
textattack: Attack time: 116.19549798965454s



False positives :95

True negative :1905

F1 score 0.7756514657980457

AUC score 0.7266637039292168



In [ ]:
print("TPrate")
print(tpr_cont5)
print("\nFPrate")
print(fpr_cont5)
print("\nF1 score")
print(f1_cont5)
print("\n AUC scores")
print(auc_cont5)

TPrate
[0.056856187290969896, 0.14332784184514002, 0.065, 0.19986023759608665, 0.11199294532627865]

FPrate
[0.0475, 0.0475, 0.0475, 0.0475, 0.0475]

F1 score
[0.909959398137091, 0.8610169491525423, 0.7054249213108684, 0.7544554455445546, 0.7756514657980457]

 AUC scores
[0.7086330292726032, 0.7251711342682433, 0.6763934862890625, 0.7716830276335958, 0.7266637039292168]


## MisMatched

### Discrete

In [ ]:
tpr_discrete = []
fpr_discrete = []
f1_discrete = []
auc_scores = []


for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-multi_nli/',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'discrete',
        gamma = None,
        freq_dict = multi_freq_dict,
        data_file='../TrainedBertModels/roberta-base-multi_nli/mismatched/attack/' + attack,
        data_name = 'mutli_nli',
        candidates =7,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_discrete.append(stat.tp/(stat.tp+stat.fn))
  fpr_discrete.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_discrete.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_scores.append(auc_score)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/roberta-base-multi_nli/


Attack From
random_nli_2000_attacked.csv




textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH discrete--


textattack: Loading nlp dataset mutli_nli, split None.
textattack: Loading nlp dataset mutli_nli, split None.
[Succeeded / Failed / Total] 1 / 2 / 3:   1%|          | 3/279 [00:00<00:14, 19.69it/s]

...

------Attacked Dataset------


[Succeeded / Failed / Total] 755 / 418 / 1173: 100%|██████████| 1173/1173 [01:10<00:00, 16.54it/s]
textattack: Attack time: 70.94003057479858s
[Succeeded / Failed / Total] 1 / 2 / 3:   0%|          | 3/2000 [00:00<02:03, 16.12it/s]


True positives :755

False_negatives :418

------Original Dataset------


[Succeeded / Failed / Total] 364 / 1636 / 2000: 100%|██████████| 2000/2000 [02:01<00:00, 16.52it/s]
textattack: Attack time: 121.05192708969116s



False positives :364

True negative :1636

F1 score 0.8071040947212629

AUC score 0.7236261639590416



In [ ]:
print("TPrate")
print(tpr_discrete)
print("\nFPrate")
print(fpr_discrete)
print("\nF1 score")
print(f1_discrete)
print("\n AUC scores")
print(auc_scores)

TPrate
[0.6164874551971327, 0.6325757575757576, 0.6316116988176727, 0.6828609986504723, 0.6436487638533674]

FPrate
[0.182, 0.182, 0.182, 0.182, 0.182]

F1 score
[0.874165108201977, 0.854308093994778, 0.7738883632923367, 0.7968826108134437, 0.8071040947212629]

 AUC scores
[0.6549914755719994, 0.6959669950571051, 0.6687712559667199, 0.747848554243363, 0.7236261639590416]


### Continuous 10%

In [ ]:
tpr_cont10 = []
fpr_cont10 = []
f1_cont10 = []
auc_cont10 = []


for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-multi_nli/',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'continuous',
        gamma = gamma_10,
        freq_dict = multi_freq_dict,
        data_file='../TrainedBertModels/roberta-base-multi_nli/mismatched/attack/' + attack,
        data_name = 'multi_nli',
        candidates =7,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_cont10.append(stat.tp/(stat.tp+stat.fn))
  fpr_cont10.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_cont10.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_cont10.append(auc_score)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/roberta-base-multi_nli/


Attack From
random_nli_2000_attacked.csv




textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH continuous--


textattack: Loading nlp dataset multi_nli, split None.
textattack: Loading nlp dataset multi_nli, split None.
[Succeeded / Failed / Total] 1 / 2 / 3:   1%|          | 3/279 [00:00<00:14, 19.45it/s]

...

------Attacked Dataset------


[Succeeded / Failed / Total] 409 / 764 / 1173: 100%|██████████| 1173/1173 [01:12<00:00, 16.23it/s]
textattack: Attack time: 72.25523066520691s
[Succeeded / Failed / Total] 1 / 2 / 3:   0%|          | 3/2000 [00:00<02:05, 15.91it/s]


True positives :409

False_negatives :764

------Original Dataset------


[Succeeded / Failed / Total] 227 / 1773 / 2000: 100%|██████████| 2000/2000 [02:02<00:00, 16.37it/s]
textattack: Attack time: 122.15843486785889s



False positives :227

True negative :1773

F1 score 0.7815737271324664

AUC score 0.7236261639590416



In [ ]:
print("TPrate")
print(tpr_cont10)
print("\nFPrate")
print(fpr_cont10)
print("\nF1 score")
print(f1_cont10)
print("\n AUC scores")
print(auc_cont10)

TPrate
[0.2903225806451613, 0.38446969696969696, 0.24517734909769756, 0.45276653171390013, 0.34867860187553285]

FPrate
[0.1135, 0.1135, 0.1135, 0.1135, 0.1135]

F1 score
[0.8929740619491311, 0.8653001464128844, 0.7111913357400722, 0.7735602094240839, 0.7815737271324664]

 AUC scores
[0.6549914755719994, 0.6959669950571051, 0.6687712559667199, 0.747848554243363, 0.7236261639590416]


## Continuous 5%

In [ ]:
tpr_cont5 = []
fpr_cont5 = []
f1_cont5 = []
auc_cont5 = []


for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-multi_nli/',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'continuous',
        gamma = gamma_5,
        freq_dict = multi_freq_dict,
        data_file='../TrainedBertModels/roberta-base-multi_nli/mismatched/attack/' + attack,
        data_name = 'multi_nli',
        candidates =7,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_cont5.append(stat.tp/(stat.tp+stat.fn))
  fpr_cont5.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_cont5.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_cont5.append(auc_score)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/roberta-base-multi_nli/


Attack From
random_nli_2000_attacked.csv




textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH continuous--


textattack: Loading nlp dataset multi_nli, split None.
textattack: Loading nlp dataset multi_nli, split None.
[Succeeded / Failed / Total] 1 / 2 / 3:   1%|          | 3/279 [00:00<00:14, 19.11it/s]

...

------Attacked Dataset------


[Succeeded / Failed / Total] 142 / 1031 / 1173: 100%|██████████| 1173/1173 [01:13<00:00, 16.00it/s]
textattack: Attack time: 73.33289742469788s
[Succeeded / Failed / Total] 1 / 2 / 3:   0%|          | 3/2000 [00:00<02:05, 15.86it/s]


True positives :142

False_negatives :1031

------Original Dataset------


[Succeeded / Failed / Total] 113 / 1887 / 2000: 100%|██████████| 2000/2000 [02:03<00:00, 16.24it/s]
textattack: Attack time: 123.12254762649536s



False positives :113

True negative :1887

F1 score 0.7673851159007726

AUC score 0.7236261639590416



In [ ]:
print("TPrate")
print(tpr_cont5)
print("\nFPrate")
print(fpr_cont5)
print("\nF1 score")
print(f1_cont5)
print("\n AUC scores")
print(auc_cont5)

TPrate
[0.08243727598566308, 0.13825757575757575, 0.06658369632856254, 0.18488529014844804, 0.12105711849957375]

FPrate
[0.0565, 0.0565, 0.0565, 0.0565, 0.0565]

F1 score
[0.9109341057204924, 0.8691847075080608, 0.7005754594393911, 0.7407262021589794, 0.7673851159007726]

 AUC scores
[0.6549914755719994, 0.6959669950571051, 0.6687712559667199, 0.747848554243363, 0.7236261639590416]


# NWS and FGWS

In [ ]:
import scipy
import torch
import pandas as pd
from collections import deque
import corpus
import tqdm
import time
import textattack
from sklearn.metrics import accuracy_score

logger = textattack.shared.logger

                               
column_encoder = {"Entailment":0, "Neutral":1, "Contradiction":2}
reversed_encoder = {0:"Entailment", 1:"Neutral", 2:"Contradiction"}

def parse_log_manager():
    # Create logger
    attack_log_manager = textattack.loggers.AttackLogManager()

    '''
    # Get current time for file naming
    timestamp = time.strftime("%Y-%m-%d-%H-%M")

    # Get default directory to save results
    current_dir = os.path.dirname(os.path.realpath(__file__))
    outputs_dir = os.path.join(
        current_dir, os.pardir, os.pardir, os.pardir, "outputs", "attacks"
    )
    out_dir_txt = out_dir_csv = os.path.normpath(outputs_dir)

    # Get default txt and csv file names
    filename_txt = f"_{timestamp}.txt"
    filename_csv = f"_{timestamp}.csv"
    
    

    # in case directory doesn't exist
    if not os.path.exists(out_dir_txt):
        os.makedirs(out_dir_txt)
    if not os.path.exists(out_dir_csv):
        os.makedirs(out_dir_csv)

    attack_log_manager.add_output_file(os.path.join(out_dir_txt, filename_txt))
    '''
    #attack_log_manager.enable_visdom()
    #attack_log_manager.enable_wandb()
    attack_log_manager.enable_stdout()
    return attack_log_manager

def evaluate_set(attacked_dataset,attack,correct_labels,logger=True):

  num_remaining_attacks = len(attacked_dataset)
  worklist = deque(range(0, len(attacked_dataset)))
  start_time = time.time()
  worklist_tail = worklist[-1]

  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)
  #pbar = tqdm.autonotebook.tqdm(total=num_remaining_attacks)
  attack_log_manager = parse_log_manager()


  num_results = 0
  num_failures = 0
  num_successes = 0
  transform_labels = []

  load_time = time.time()

  probabilities = []
  for result in attack.attack_dataset(attacked_dataset, indices=worklist):

    #or_label = result.original_result.output
    #initial_score = result.original_result.raw_output[or_label]
    #pertubated_score = result.perturbed_result.raw_output[or_label]
    initial_score = 1-result.original_result.score
    pertubated_score = 1-result.perturbed_result.score
    probabilities.append(initial_score-pertubated_score)
    transform_labels.append(result.perturbed_result.output)

    #if logger:
    #  attack_log_manager.log_result(result)

    #  print("\n")
    pbar.update(1)
    
    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )
    
  pbar.close()

  # Enable summary stdout
  
  if logger:
    attack_log_manager.log_summary()
    attack_log_manager.flush()
  print()
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  return num_successes, num_failures, probabilities, transform_labels



class EvalDefenseModel():

  def __init__(
      self,
      defense_from_class,
      model_from_file=None, 
      model=None,
      model_from_huggingface=None,
      model_batch_size=32,
      delta = 10,
      gamma = 1,
      type = 'discrete',
      freq_dict = None,
      data_file = None,
      data_name = "multi_nli",
      candidates = 7,
      logger = True,
      only_attack = False,
      only_original = False,
      #indices_to_modify=None,
      #shifted_idxs=True,
    ):

    # Get attack
    attack = parse_defense_from_class(defense_from_class,freq_dict,delta,type,gamma,candidates,model_from_file, model,model_from_huggingface,model_batch_size)
    
    #Get datasets
    attacked_data = pd.read_csv(data_file,usecols=["premise",'attacked_hypothesis', 'label', 'modified_indices' , "attacked_label"])
    original_data = pd.read_csv(data_file,usecols=["premise",'original_hypothesis', "original_label"])
    or_labels = original_data["original_label"]
    

    to_remove = []
    for index,x in attacked_data.iterrows():
      if x['modified_indices'] == '[]':
        to_remove.append(index)
    attacked_data = attacked_data.drop(to_remove)
    correct_labels = attacked_data['label']
    
    

    attacked_data = attacked_data.drop(['modified_indices','label'],axis=1)

    attacked_data["attacked_label"] = [column_encoder[i] for i in attacked_data["attacked_label"] ]
    # Get original adv accuracy 
    self.adv_acc = accuracy_score(correct_labels,attacked_data["attacked_label"])

    original_data["original_label"] = [column_encoder[i] for i in original_data["original_label"] ]

    attacked_data = attacked_data.rename(columns={'attacked_hypothesis': 'hypothesis','attacked_label':'label'})
    attacked_dataset = nlp.Dataset.from_pandas(attacked_data)

    attacked_dataset = textattack.datasets.HuggingFaceNlpDataset(
            name=data_name, shuffle=False, split=None, dataset=attacked_dataset, label_map = reversed_encoder)
    
    original_data = original_data.rename(columns={'original_hypothesis': 'hypothesis','original_label':'label'})
    original_dataset = nlp.Dataset.from_pandas(original_data)

    original_dataset = textattack.datasets.HuggingFaceNlpDataset(
            name=data_name, shuffle=False, split=None, dataset=original_dataset, label_map = reversed_encoder)
    
    print("------Attacked Dataset------")
    if not only_original:
      true_positives, false_negatives, pert_probs, transform_labels = evaluate_set(attacked_dataset,attack,logger)
      self.tp = true_positives
      self.fn = false_negatives
      self.adv_probs = pert_probs

      print("True positives :" + str(true_positives) + "\n")
      print("False_negatives :" + str(false_negatives) + "\n")

      # Get transformed adv accuracy 
      self.transform_acc = accuracy_score(correct_labels,transform_labels)

    if  not only_attack:
      print("------Original Dataset------")
      false_positives, true_negatives, or_probs, _i10 = evaluate_set(original_dataset,attack,logger)
      print("False positives :" + str(false_positives) + "\n")
      print("True negative :" + str(true_negatives) + "\n")
      self.fp = false_positives
      self.tn = true_negatives
      self.or_probs = or_probs

#Matched

In [ ]:
tpr_nws = []
fpr_nws = []
f1_nws = []
auc_nws = []
adv_acc_nws = []
tr_acc_nws =[]


for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "NWS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-multi_nli/',
        model_batch_size=32,
        delta = None,
        type = None,
        gamma = None,
        freq_dict = multi_freq_dict,
        data_file='../TrainedBertModels/roberta-base-multi_nli/matched/attack/' + attack,
        data_name = 'mutli_nli',
        candidates =7,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_nws.append(stat.tp/(stat.tp+stat.fn))
  fpr_nws.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_nws.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_nws.append(auc_score)

  adv_acc_nws.append(stat.adv_acc) 
  tr_acc_nws.append(stat.transform_acc)
  print("Adversarial Acc " + str(stat.adv_acc))
  print("Transform Acc " + str(stat.transform_acc))


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print("TP rates")
print(tpr_nws)
print("\nFP rates")
print(fpr_nws)
print("\nF1 scores")
print(f1_nws)
print("\nAuc scores")
print(auc_nws)
print("\nAdv_acc")
print(adv_acc_nws)
print("\nTransform_acc")
print(tr_acc_nws)

TP rates
[0.056511056511056514, 0.10303587856485741, 0.11072664359861592, 0.14426992437463643, 0.08488372093023255]

FP rates
[0.002, 0.0035, 0.002, 0.003, 0.0035]

F1 scores
[0.7754467754467754, 0.8023349436392915, 0.7208378475984109, 0.7297346752058554, 0.7160050296389439]

Auc scores
[0.5039025215968705, 0.5107085652636453, 0.5501437164172949, 0.5595793800662683, 0.5228443445781503]

Adv_acc
[0.7551187551187551, 0.44158233670653174, 0.07727797001153403, 0.16753926701570682, 0.34069767441860466]

Transform_acc
[0.7903357903357904, 0.5335786568537259, 0.18685121107266436, 0.30017452006980805, 0.413953488372093]


In [ ]:
tpr_fgws = []
fpr_fgws = []
f1_fgws = []
auc_fgws = []
adv_acc_fgws = []
tr_acc_fgws = []

for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-multi_nli/',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'discrete',
        gamma = None,
        freq_dict = multi_freq_dict,
        data_file='../TrainedBertModels/roberta-base-multi_nli/matched/attack/' + attack,
        data_name = 'multi_nli',
        candidates =7,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_fgws.append(stat.tp/(stat.tp+stat.fn))
  fpr_fgws.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_fgws.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_fgws.append(auc_score)
  
  adv_acc_fgws.append(stat.adv_acc) 
  tr_acc_fgws.append(stat.transform_acc)
  print("Adversarial Acc " + str(stat.adv_acc))
  print("Transform Acc " + str(stat.transform_acc))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print("TP rates")
print(tpr_fgws)
print("\nFP rates")
print(fpr_fgws)
print("\nF1 scores")
print(f1_fgws)
print("\nAuc scores")
print(auc_fgws)
print("\nAdv_acc")
print(adv_acc_fgws)
print("\nTransform_acc")
print(tr_acc_fgws)

TP rates
[0.26371826371826373, 0.43514259429622815, 0.5778546712802768, 0.5683536940081443, 0.436046511627907]

FP rates
[0.1725, 0.1725, 0.1725, 0.1725, 0.1725]

F1 scores
[0.7268335529205096, 0.7753572265167487, 0.7545019375427399, 0.7527859904480328, 0.7156756756756757]

Auc scores
[0.4977615971455038, 0.5630328735351046, 0.6453608746935235, 0.6614355039203563, 0.5582438046410222]

Adv_acc
[0.7551187551187551, 0.44158233670653174, 0.07727797001153403, 0.16753926701570682, 0.34069767441860466]

Transform_acc
[0.7968877968877969, 0.7056117755289788, 0.5778546712802768, 0.6620127981384526, 0.6773255813953488]


#Mismatched

In [ ]:
tpr_nws = []
fpr_nws = []
f1_nws = []
auc_nws = []
adv_acc_nws = []
tr_acc_nws =[]


for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "NWS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-multi_nli/',
        model_batch_size=32,
        delta = None,
        type = None,
        gamma = None,
        freq_dict = multi_freq_dict,
        data_file='../TrainedBertModels/roberta-base-multi_nli/mismatched/attack/' + attack,
        data_name = 'mutli_nli',
        candidates =7,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_nws.append(stat.tp/(stat.tp+stat.fn))
  fpr_nws.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_nws.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_nws.append(auc_score)

  adv_acc_nws.append(stat.adv_acc) 
  tr_acc_nws.append(stat.transform_acc)
  print("Adversarial Acc " + str(stat.adv_acc))
  print("Transform Acc " + str(stat.transform_acc))


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print("TP rates")
print(tpr_nws)
print("\nFP rates")
print(fpr_nws)
print("\nF1 scores")
print(f1_nws)
print("\nAuc scores")
print(auc_nws)
print("\nAdv_acc")
print(adv_acc_nws)
print("\nTransform_acc")
print(tr_acc_nws)

TP rates
[0.0688550840672538, 0.09831460674157304, 0.1313073394495413, 0.15747126436781608, 0.10989010989010989]

FP rates
[0.0065, 0.0095, 0.008, 0.0095, 0.0075]

F1 scores
[0.7716504854368933, 0.8013754045307444, 0.7215857428623387, 0.7273728657976868, 0.718682114409848]

Auc scores
[0.5094176332579274, 0.49987481282175367, 0.5593089551627861, 0.5678335754723213, 0.539824766034854]

Adv_acc
[0.77662129703763, 0.5056179775280899, 0.07855504587155963, 0.1482758620689655, 0.3215731636784268]

Transform_acc
[0.8022417934347478, 0.5823970037453183, 0.2029816513761468, 0.29482758620689653, 0.40948525159051474]


In [ ]:
tpr_fgws = []
fpr_fgws = []
f1_fgws = []
auc_fgws = []
adv_acc_fgws = []
tr_acc_fgws = []

for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-multi_nli/',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'discrete',
        gamma = None,
        freq_dict = multi_freq_dict,
        data_file='../TrainedBertModels/roberta-base-multi_nli/mismatched/attack/' + attack,
        data_name = 'multi_nli',
        candidates =7,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_fgws.append(stat.tp/(stat.tp+stat.fn))
  fpr_fgws.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_fgws.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_fgws.append(auc_score)
  
  adv_acc_fgws.append(stat.adv_acc) 
  tr_acc_fgws.append(stat.transform_acc)
  print("Adversarial Acc " + str(stat.adv_acc))
  print("Transform Acc " + str(stat.transform_acc))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print("TP rates")
print(tpr_fgws)
print("\nFP rates")
print(fpr_fgws)
print("\nF1 scores")
print(f1_fgws)
print("\nAuc scores")
print(auc_fgws)
print("\nAdv_acc")
print(adv_acc_fgws)
print("\nTransform_acc")
print(tr_acc_fgws)

TP rates
[0.2722177742193755, 0.3951310861423221, 0.5837155963302753, 0.6, 0.459224985540775]

FP rates
[0.1935, 0.1935, 0.1935, 0.1935, 0.1935]

F1 scores
[0.7134011499336577, 0.7574548015966188, 0.7434892832449873, 0.7486655836621026, 0.7093227792436234]

Auc scores
[0.4913544778176424, 0.5261479712858926, 0.6185082203860155, 0.6696411585083895, 0.558855095183347]

Adv_acc
[0.77662129703763, 0.5056179775280899, 0.07855504587155963, 0.1482758620689655, 0.3215731636784268]

Transform_acc
[0.7646116893514812, 0.7088014981273408, 0.595756880733945, 0.6752873563218391, 0.6813186813186813]
